In [1]:
import os
import pandas as pd
from strsimpy.levenshtein import Levenshtein
from strsimpy.normalized_levenshtein import NormalizedLevenshtein
from valentine import valentine_match, valentine_metrics
from valentine.algorithms import Coma, Cupid, DistributionBased, JaccardLevenMatcher, SimilarityFlooding


In [2]:
mapping_columns = {
    "name": ["model", "name", "model-split", "Tên"],
    "url": ["url", "short-link", "link"],
    "cpu": ["chipset", "Bộ xử lý trung tâm", "chip_set", "chip", "chip xử lý"],
    "os": ["os", "hệ điều hành"],
    "gpu": ["Chip đồ họa", "gpu", "chip_do_hoa"],
    "ram": ["ram", "RAM", "Ram"],
    "rom": ["Bộ nhớ trong", "rom", "rom-split"],
    "price": ["giá", "price"],
    "connect": ["kết nối", "connect"],
    "screentype": ["công nghệ màn hình", "loại màn hình", "display_type", "chất lượng màn hình", "kiểu màn hình"],
    "color": ["color", "color-split", "màu"],
    "front_camera": ["camera phụ", "camera_truoc"],
    "rear_camera": ["camera chính", "camera_sau"],
    "battery_capacity": ["dung lượng pin", "pin"],
    "bluetooth": ["bluetooth"],
    "wifi": ["wifi", "Wifi"],
    "size": ["màn hình rộng", "kích thước màn hình"],
    "resolution": ["resolution", "độ phân giải", "độ phân giải màn hình"],
    "weight": ["trọng lượng", "product_weight", "cân nặng", "Kích thước khối lượng"],
    "jack_audio": ["jack tai nghe"],
    "mobile_network": ["mạng di động", "hỗ trợ 5G"],
    "sim": ["sim"],
    "security": ["bảo mật nâng cao", "security"],
    "material": ["material", "chất liệu"]
}
target_schema = []
for col in mapping_columns:
    target_schema.append(col)

In [3]:
path = "../PreprocessingData/Dataprocess/iphone/ddtm.json"
df = pd.read_json(path)

In [9]:
list_columns = list(df.columns)
columns = list_columns[0:43]

In [13]:

df2 = df[columns].drop([" Màn hình rộng "], axis=1)

In [ ]:
columns = list(df2.columns)
columns

In [16]:
levenshtein = Levenshtein()

def similarity_score(x, y):
    x = x.lower().strip()
    y = y.lower().strip()
    return (len(x) + len(y) - levenshtein.distance(x, y)) / (len(x) + len(y))
def check_similarity(x, y, threshold=0.8):
    score = similarity_score(x, y)
    if score > threshold:
        return True
    return False 

In [17]:
threshold = 0.85

res = dict()
for colum in columns:
    for key in mapping_columns:
        listMatching = mapping_columns[key]
        for match in listMatching:
            score = similarity_score(colum, match)
            if(score > threshold):
                print("{0:20}\t{0:20}\t".format(colum, match) + str(score))
                res[colum] = key
        

Bluetooth           	Bluetooth           	1.0
Bảo mật nâng cao    	Bảo mật nâng cao    	1.0
Bộ nhớ trong        	Bộ nhớ trong        	1.0
Chip xử lý          	Chip xử lý          	1.0
Chip đồ họa         	Chip đồ họa         	1.0
Công nghệ màn hình  	Công nghệ màn hình  	1.0
Dung lượng pin      	Dung lượng pin      	1.0
Jack tai nghe       	Jack tai nghe       	1.0
Kích thước khối lượng 	Kích thước khối lượng 	1.0
Model-split         	Model-split         	1.0
Màn hình rộng       	Màn hình rộng       	1.0
Mạng di động        	Mạng di động        	1.0
RAM                 	RAM                 	1.0
RAM                 	RAM                 	1.0
RAM                 	RAM                 	1.0
Rom-split           	Rom-split           	1.0
Sim                 	Sim                 	1.0
Wifi                	Wifi                	1.0
Wifi                	Wifi                	1.0
color               	color               	1.0
name                	name                	1.0
price               	price    

In [18]:
res

{'Bluetooth ': 'bluetooth',
 'Bảo mật nâng cao ': 'security',
 'Bộ nhớ trong ': 'rom',
 'Chip xử lý ': 'cpu',
 'Chip đồ họa ': 'gpu',
 'Công nghệ màn hình ': 'screentype',
 'Dung lượng pin ': 'battery_capacity',
 'Jack tai nghe ': 'jack_audio',
 'Kích thước khối lượng ': 'weight',
 'Model-split': 'name',
 'Màn hình rộng ': 'size',
 'Mạng di động ': 'mobile_network',
 'RAM ': 'ram',
 'Rom-split': 'rom',
 'Sim ': 'sim',
 'Wifi ': 'wifi',
 'color': 'color',
 'name': 'name',
 'price': 'price',
 'url': 'url',
 'Độ phân giải ': 'resolution'}

In [19]:

data = pd.DataFrame(columns=target_schema)

In [20]:
tmp = res.pop("name")
tmp2 = res.pop("Bộ nhớ trong ")
res


{'Bluetooth ': 'bluetooth',
 'Bảo mật nâng cao ': 'security',
 'Chip xử lý ': 'cpu',
 'Chip đồ họa ': 'gpu',
 'Công nghệ màn hình ': 'screentype',
 'Dung lượng pin ': 'battery_capacity',
 'Jack tai nghe ': 'jack_audio',
 'Kích thước khối lượng ': 'weight',
 'Model-split': 'name',
 'Màn hình rộng ': 'size',
 'Mạng di động ': 'mobile_network',
 'RAM ': 'ram',
 'Rom-split': 'rom',
 'Sim ': 'sim',
 'Wifi ': 'wifi',
 'color': 'color',
 'price': 'price',
 'url': 'url',
 'Độ phân giải ': 'resolution'}

In [23]:
matched_col = []
source_col = []
for key in res:
    matched_col.append(res[key])
    source_col.append(key)

In [24]:
source_col

['Bluetooth ',
 'Bảo mật nâng cao ',
 'Chip xử lý ',
 'Chip đồ họa ',
 'Công nghệ màn hình ',
 'Dung lượng pin ',
 'Jack tai nghe ',
 'Kích thước khối lượng ',
 'Model-split',
 'Màn hình rộng ',
 'Mạng di động ',
 'RAM ',
 'Rom-split',
 'Sim ',
 'Wifi ',
 'color',
 'price',
 'url',
 'Độ phân giải ']

In [25]:
df2_tmp = df2[source_col]
df2_tmp = df2_tmp.rename(columns=res)
df2_tmp.head(5)

,bluetooth,security,cpu,gpu,screentype,battery_capacity,jack_audio,weight,name,size,mobile_network,ram,rom,sim,wifi,color,price,url,resolution
0,"A2DP, v5.0",Mở khoá khuôn mặt Face ID,Apple A13 Bionic 6 nhân,Apple GPU 4 nhân,IPS LCD,3110 mAh,Lightning,Dài 150.9 mm - Ngang 75.7 mm - Dày 8.3 mm - Nặ...,IPHONE 11,6.1 - Tần số quét 60 Hz,Hỗ trợ 4G,4 GB,64GB,1 Nano SIM &amp; 1 eSIM,Wi-Fi hotspot Wi-Fi 802.11 a/b/g/n/ac/ax Dual...,Xanh lá,11.640.000đ,https://didongthongminh.vn/iphone-11-64gb,828 x 1792 Pixels
1,"A2DP, v5.0",Mở khoá khuôn mặt Face ID,Apple A13 Bionic 6 nhân,Apple GPU 4 nhân,IPS LCD,3110 mAh,Lightning,Dài 150.9 mm - Ngang 75.7 mm - Dày 8.3 mm - Nặ...,IPHONE 11,6.1 - Tần số quét 60 Hz,Hỗ trợ 4G,4 GB,64GB,1 Nano SIM &amp; 1 eSIM,Wi-Fi hotspot Wi-Fi 802.11 a/b/g/n/ac/ax Dual...,Đen,11.640.000đ,https://didongthongminh.vn/iphone-11-64gb,828 x 1792 Pixels
2,"A2DP, v5.0",Mở khoá khuôn mặt Face ID,Apple A13 Bionic 6 nhân,Apple GPU 4 nhân,IPS LCD,3110 mAh,Lightning,Dài 150.9 mm - Ngang 75.7 mm - Dày 8.3 mm - Nặ...,IPHONE 11,6.1 - Tần số quét 60 Hz,Hỗ trợ 4G,4 GB,64GB,1 Nano SIM &amp; 1 eSIM,Wi-Fi hotspot Wi-Fi 802.11 a/b/g/n/ac/ax Dual...,Tím,11.640.000đ,https://didongthongminh.vn/iphone-11-64gb,828 x 1792 Pixels
3,"A2DP, v5.0",Mở khoá khuôn mặt Face ID,Apple A13 Bionic 6 nhân,Apple GPU 4 nhân,IPS LCD,3110 mAh,Lightning,Dài 150.9 mm - Ngang 75.7 mm - Dày 8.3 mm - Nặ...,IPHONE 11,6.1 - Tần số quét 60 Hz,Hỗ trợ 4G,4 GB,64GB,1 Nano SIM &amp; 1 eSIM,Wi-Fi hotspot Wi-Fi 802.11 a/b/g/n/ac/ax Dual...,Đỏ,11.890.000đ,https://didongthongminh.vn/iphone-11-64gb,828 x 1792 Pixels
4,"A2DP, v5.0",Mở khoá khuôn mặt Face ID,Apple A13 Bionic 6 nhân,Apple GPU 4 nhân,IPS LCD,3110 mAh,Lightning,Dài 150.9 mm - Ngang 75.7 mm - Dày 8.3 mm - Nặ...,IPHONE 11,6.1 - Tần số quét 60 Hz,Hỗ trợ 4G,4 GB,64GB,1 Nano SIM &amp; 1 eSIM,Wi-Fi hotspot Wi-Fi 802.11 a/b/g/n/ac/ax Dual...,Trắng,11.390.000đ,https://didongthongminh.vn/iphone-11-64gb,828 x 1792 Pixels


In [26]:
df2_tmp.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1584 entries, 0 to 1583
Data columns (total 19 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   bluetooth         1418 non-null   object
 1   security          1418 non-null   object
 2   cpu               1283 non-null   object
 3   gpu               1400 non-null   object
 4   screentype        1418 non-null   object
 5   battery_capacity  1418 non-null   object
 6   jack_audio        1402 non-null   object
 7   weight            1418 non-null   object
 8   name              1584 non-null   object
 9   size              1418 non-null   object
 10  mobile_network    1402 non-null   object
 11  ram               1328 non-null   object
 12  rom               1572 non-null   object
 13  sim               1418 non-null   object
 14  wifi              1418 non-null   object
 15  color             1584 non-null   object
 16  price             1584 non-null   object
 17  url           

In [27]:
data = pd.DataFrame(columns=target_schema)
data

,name,url,cpu,os,gpu,ram,rom,price,connect,screentype,...,bluetooth,wifi,size,resolution,weight,jack_audio,mobile_network,sim,security,material


In [29]:
data[matched_col] = df2_tmp

In [31]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1584 entries, 0 to 1583
Data columns (total 24 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   name              1584 non-null   object
 1   url               1584 non-null   object
 2   cpu               1283 non-null   object
 3   os                0 non-null      object
 4   gpu               1400 non-null   object
 5   ram               1328 non-null   object
 6   rom               1572 non-null   object
 7   price             1584 non-null   object
 8   connect           0 non-null      object
 9   screentype        1418 non-null   object
 10  color             1584 non-null   object
 11  front_camera      0 non-null      object
 12  rear_camera       0 non-null      object
 13  battery_capacity  1418 non-null   object
 14  bluetooth         1418 non-null   object
 15  wifi              1418 non-null   object
 16  size              1418 non-null   object
 17  resolution    

In [32]:
data.to_json("./IntegratedData/target_data.json")

In [33]:
data 

,name,url,cpu,os,gpu,ram,rom,price,connect,screentype,...,bluetooth,wifi,size,resolution,weight,jack_audio,mobile_network,sim,security,material
0,IPHONE 11,https://didongthongminh.vn/iphone-11-64gb,Apple A13 Bionic 6 nhân,NaN,Apple GPU 4 nhân,4 GB,64GB,11.640.000đ,NaN,IPS LCD,...,"A2DP, v5.0",Wi-Fi hotspot Wi-Fi 802.11 a/b/g/n/ac/ax Dual...,6.1 - Tần số quét 60 Hz,828 x 1792 Pixels,Dài 150.9 mm - Ngang 75.7 mm - Dày 8.3 mm - Nặ...,Lightning,Hỗ trợ 4G,1 Nano SIM &amp; 1 eSIM,Mở khoá khuôn mặt Face ID,NaN
1,IPHONE 11,https://didongthongminh.vn/iphone-11-64gb,Apple A13 Bionic 6 nhân,NaN,Apple GPU 4 nhân,4 GB,64GB,11.640.000đ,NaN,IPS LCD,...,"A2DP, v5.0",Wi-Fi hotspot Wi-Fi 802.11 a/b/g/n/ac/ax Dual...,6.1 - Tần số quét 60 Hz,828 x 1792 Pixels,Dài 150.9 mm - Ngang 75.7 mm - Dày 8.3 mm - Nặ...,Lightning,Hỗ trợ 4G,1 Nano SIM &amp; 1 eSIM,Mở khoá khuôn mặt Face ID,NaN
2,IPHONE 11,https://didongthongminh.vn/iphone-11-64gb,Apple A13 Bionic 6 nhân,NaN,Apple GPU 4 nhân,4 GB,64GB,11.640.000đ,NaN,IPS LCD,...,"A2DP, v5.0",Wi-Fi hotspot Wi-Fi 802.11 a/b/g/n/ac/ax Dual...,6.1 - Tần số quét 60 Hz,828 x 1792 Pixels,Dài 150.9 mm - Ngang 75.7 mm - Dày 8.3 mm - Nặ...,Lightning,Hỗ trợ 4G,1 Nano SIM &amp; 1 eSIM,Mở khoá khuôn mặt Face ID,NaN
3,IPHONE 11,https://didongthongminh.vn/iphone-11-64gb,Apple A13 Bionic 6 nhân,NaN,Apple GPU 4 nhân,4 GB,64GB,11.890.000đ,NaN,IPS LCD,...,"A2DP, v5.0",Wi-Fi hotspot Wi-Fi 802.11 a/b/g/n/ac/ax Dual...,6.1 - Tần số quét 60 Hz,828 x 1792 Pixels,Dài 150.9 mm - Ngang 75.7 mm - Dày 8.3 mm - Nặ...,Lightning,Hỗ trợ 4G,1 Nano SIM &amp; 1 eSIM,Mở khoá khuôn mặt Face ID,NaN
4,IPHONE 11,https://didongthongminh.vn/iphone-11-64gb,Apple A13 Bionic 6 nhân,NaN,Apple GPU 4 nhân,4 GB,64GB,11.390.000đ,NaN,IPS LCD,...,"A2DP, v5.0",Wi-Fi hotspot Wi-Fi 802.11 a/b/g/n/ac/ax Dual...,6.1 - Tần số quét 60 Hz,828 x 1792 Pixels,Dài 150.9 mm - Ngang 75.7 mm - Dày 8.3 mm - Nặ...,Lightning,Hỗ trợ 4G,1 Nano SIM &amp; 1 eSIM,Mở khoá khuôn mặt Face ID,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1579,IPHONE 11,https://didongthongminh.vn/iphone-11-128gb-chi...,Apple A13 Bionic 6 nhân,NaN,Apple GPU 4 nhân,4 GB,128GB,13.190.000đ,NaN,IPS LCD,...,A2DPv5.0,Wi-Fi hotspotWi-Fi 802.11 a/b/g/n/ac/axDual-ba...,6.1 - Tần số quét 60 Hz,828 x 1792 Pixels,Dài 150.9 mm - Ngang 75.7 mm - Dày 8.3 mm - Nặ...,Lightning,Hỗ trợ 4G,1 Nano SIM &amp; 1 eSIM,Mở khoá khuôn mặt Face ID,NaN
1580,IPHONE 11,https://didongthongminh.vn/iphone-11-128gb-chi...,Apple A13 Bionic 6 nhân,NaN,Apple GPU 4 nhân,4 GB,128GB,13.190.000đ,NaN,IPS LCD,...,A2DPv5.0,Wi-Fi hotspotWi-Fi 802.11 a/b/g/n/ac/axDual-ba...,6.1 - Tần số quét 60 Hz,828 x 1792 Pixels,Dài 150.9 mm - Ngang 75.7 mm - Dày 8.3 mm - Nặ...,Lightning,Hỗ trợ 4G,1 Nano SIM &amp; 1 eSIM,Mở khoá khuôn mặt Face ID,NaN
1581,IPHONE 11,https://didongthongminh.vn/iphone-11-128gb-chi...,Apple A13 Bionic 6 nhân,NaN,Apple GPU 4 nhân,4 GB,128GB,13.290.000đ,NaN,IPS LCD,...,A2DPv5.0,Wi-Fi hotspotWi-Fi 802.11 a/b/g/n/ac/axDual-ba...,6.1 - Tần số quét 60 Hz,828 x 1792 Pixels,Dài 150.9 mm - Ngang 75.7 mm - Dày 8.3 mm - Nặ...,Lightning,Hỗ trợ 4G,1 Nano SIM &amp; 1 eSIM,Mở khoá khuôn mặt Face ID,NaN
1582,IPHONE 11,https://didongthongminh.vn/iphone-11-128gb-chi...,Apple A13 Bionic 6 nhân,NaN,Apple GPU 4 nhân,4 GB,128GB,13.190.000đ,NaN,IPS LCD,...,A2DPv5.0,Wi-Fi hotspotWi-Fi 802.11 a/b/g/n/ac/axDual-ba...,6.1 - Tần số quét 60 Hz,828 x 1792 Pixels,Dài 150.9 mm - Ngang 75.7 mm - Dày 8.3 mm - Nặ...,Lightning,Hỗ trợ 4G,1 Nano SIM &amp; 1 eSIM,Mở khoá khuôn mặt Face ID,NaN
